In [1]:
import os, sys
parent_dir = os.path.abspath('../..')

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Import

In [2]:
import time
from itertools import cycle
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.utils import make_grid
import torchvision.datasets as dset
import matplotlib.pyplot as plt

import utils_os
import mi


torch.manual_seed(time.time())

%load_ext autoreload
%autoreload

# Global Variables

In [3]:
DEVICE = 'cuda:0' 

class Hyperparams(utils_os.ConfigDict):

    def __init__(self): 
        self.device = DEVICE
        self.learning_rate = 1e-3
        self.n_epochs = 100
        self.grad_clip = None
        self.batch_size = 128
        self.image_shape = (32, 32)
        self.update_inner_every_num_epoch = 2

BASE_HYPERPARAMS = Hyperparams()

ROOT = '../..'
MODEL_SAVE_DIR = f'{ROOT}/results/c_s'
DATASET_DIR = f'{ROOT}/data'

NUM_COLOR_CHANNELS = 3
NUM_CLASSES = 9
NUM_DOMAIN = 2

DIM_Z_CONTENT, DIM_Z_DOMAIN = 128, 128

INFOMIN_TRAIN_BATCH_SIZE = 2500

# Prepare Dataloaders

### Align labels

In [4]:
"""
CIFAR
0: plane
1: car
2: bird
3: cat
4: deer
5: dog
6: frog
7: horse
8: ship
9: truck

remove frog(6)
"""

"""
STL
0: plane
1: bird
2: car
3: cat
4: deer
5: dog
6: horse
7: monkey
8: ship
9: truck

remove monkey(7)
"""
CIFAR_MAP = {
    0: 0,
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: -2,
    7: 6,
    8: 7,
    9: 8,
    -1: -1,
}

STL_MAP = {
    0: 0,
    1: 2,
    2: 1,
    3: 3,
    4: 4,
    5: 5,
    6: 6,
    7: -2,
    8: 7,
    9: 8,
    -1: -1,
}

### Load CIFAR10 and STL

In [5]:
if not os.path.exists(DATASET_DIR):
    os.mkdir(DATASET_DIR)
    
# define data transformations
trans_train = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomCrop(32, padding=4, padding_mode='reflect'), 
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
])
trans_test = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(), 
])
target_trans_cifar = transforms.Compose([lambda x: CIFAR_MAP[x]])
target_trans_stl = transforms.Compose([lambda x: STL_MAP[x]])


# load cifar (remove the class 'frog')
m_train_set = dset.CIFAR10(root=DATASET_DIR, train=True, transform=trans_train, target_transform=target_trans_cifar, download=True)
m_test_set = dset.CIFAR10(root=DATASET_DIR, train=False, transform=trans_test, target_transform=target_trans_cifar, download=True)

cifar_train_indices = [idx for idx, (_, target) in enumerate(m_train_set) if target != -2]
m_train_loader = torch.utils.data.DataLoader(
                 dataset=torch.utils.data.Subset(m_train_set, cifar_train_indices),
                 batch_size=BASE_HYPERPARAMS.batch_size)
cifar_test_indices = [idx for idx, (_, target) in enumerate(m_test_set) if target != -2]
m_test_loader = torch.utils.data.DataLoader(
                dataset=torch.utils.data.Subset(m_test_set, cifar_test_indices),
                batch_size=BASE_HYPERPARAMS.batch_size,
                shuffle=False)
domain1 = 'CIFAR'

# load stl-10 (remove the class 'monkey')
mm_train_set = dset.STL10(root=DATASET_DIR, split='train', transform=trans_train, target_transform=target_trans_stl, download=True)
mm_test_set = dset.STL10(root=DATASET_DIR, split='test', transform=trans_test, target_transform=target_trans_stl, download=True)

stl_train_indices = [idx for idx, (_, target) in enumerate(mm_train_set) if target != -2]
mm_train_loader = torch.utils.data.DataLoader(
                 dataset=torch.utils.data.Subset(mm_train_set, stl_train_indices),
                 batch_size=BASE_HYPERPARAMS.batch_size)
stl_test_indices = [idx for idx, (_, target) in enumerate(mm_test_set) if target != -2]
mm_test_loader = torch.utils.data.DataLoader(
                dataset=torch.utils.data.Subset(mm_test_set, stl_test_indices),
                batch_size=BASE_HYPERPARAMS.batch_size,
                shuffle=False)  
domain2 = 'STL'

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


### Load all data into memeory for faster access when training infomin layers

In [6]:
def loader2tensor(dataloader):
    all_data, all_target = [], []
    for data, target in iter(dataloader):
        all_data.append(data)
        all_target.append(target)
    print('loader2tensor finished.', 'data max=', data.max(), 'data min=', data.min())
    return torch.cat(all_data, dim=0), torch.cat(all_target, dim=0)

all_data_m, all_label_m = loader2tensor(m_train_loader)
all_data_test_m, all_label_test_m = loader2tensor(m_test_loader)

all_data_mm, all_label_mm = loader2tensor(mm_train_loader)
all_data_test_mm, all_label_test_mm = loader2tensor(mm_test_loader)

loader2tensor finished. data max= tensor(1.) data min= tensor(0.)
loader2tensor finished. data max= tensor(1.) data min= tensor(0.)
loader2tensor finished. data max= tensor(1.) data min= tensor(0.)
loader2tensor finished. data max= tensor(1.) data min= tensor(0.)


# Experiments

In [7]:
from tasks.domain_adaptation.experiment import DomainAdaptation, train, test
from train_test_routine import exp_run


def inner_batch_provider():
    idx1, idx2 = torch.randperm(len(all_data_m)), torch.randperm(len(all_data_mm))
    infomin_x1, infomin_x2 = all_data_m[idx1[:INFOMIN_TRAIN_BATCH_SIZE]].to(DEVICE), all_data_mm[idx2[:INFOMIN_TRAIN_BATCH_SIZE]].to(DEVICE)

    return infomin_x1, infomin_x2


def get_model_path(hyperparams, epoch=None):
    if epoch is not None:
        return f'{MODEL_SAVE_DIR}/{hyperparams.estimator}/model_{hyperparams.get_name("alpha", "beta", "gamma")}_{epoch}'
    return f'{MODEL_SAVE_DIR}/{hyperparams.estimator}/model_{hyperparams.get_name("alpha", "beta", "gamma")}'

In [10]:
import copy

hyperparams = copy.deepcopy(BASE_HYPERPARAMS)
hyperparams.alpha = 1.0
hyperparams.beta = 1.0
hyperparams.gamma = 0.3
hyperparams.n_slice = 100
hyperparams.learning_rate = 1e-3
hyperparams.batch_size = 128
hyperparams.estimator = 'SLICE'
hyperparams.n_epochs = 70

da = DomainAdaptation(DIM_Z_CONTENT, DIM_Z_DOMAIN, NUM_CLASSES, NUM_DOMAIN, hyperparams).to(DEVICE)

da = exp_run(
    (m_train_loader, mm_train_loader), (m_test_loader, mm_test_loader),
    train, test,
    inner_batch_provider, get_model_path,
    hyperparams,
    device='cuda:0',
    model=da,
    # scheduler_func=lambda optimizer: torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20)
    scheduler_func=lambda optimizer: torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
)

epoch: 0, loss: 1.7817467652576071, loss_content: 1.0852251606927792, loss_domain: 0.6799803065582061 redundancy: 0.055137630318402985  acc_d1: 61.477779388427734 acc_d2: 53.08051681518555 acc_domain: 56.552913665771484

epoch: 1, loss: 1.3305103661308826, loss_content: 0.7113589728382271, loss_domain: 0.6029036729268624 redundancy: 0.05415910342052369  acc_d1: 74.74444580078125 acc_d2: 65.25800323486328 acc_domain: 67.73565673828125

epoch: 2, loss: 1.1524168370475232, loss_content: 0.7489599266522368, loss_domain: 0.3802866287214655 redundancy: 0.07723425218546895  acc_d1: 73.63333129882812 acc_d2: 59.986656188964844 acc_domain: 88.10026550292969

epoch: 3, loss: 1.0210860724180517, loss_content: 0.6229801727852351, loss_domain: 0.38215491301576854 redundancy: 0.0531699749916582  acc_d1: 78.72222137451172 acc_d2: 66.95951843261719 acc_domain: 88.44486236572266

epoch: 4, loss: 0.8691306844563551, loss_content: 0.5297532916908533, loss_domain: 0.32583757668313845 redundancy: 0.0451327

In [10]:
import copy

hyperparams = copy.deepcopy(BASE_HYPERPARAMS)
hyperparams.alpha = 1.0
hyperparams.beta = 1.0
hyperparams.gamma = 0.005
hyperparams.estimator = 'CLUB'
hyperparams.learning_rate = 1e-3
hyperparams.inner_lr = 1e-3
hyperparams.inner_epochs = 5
hyperparams.inner_batch_size = 2500

da = DomainAdaptation(DIM_Z_CONTENT, DIM_Z_DOMAIN, NUM_CLASSES, NUM_DOMAIN, hyperparams).to(DEVICE)

da = exp_run(
    (m_train_loader, mm_train_loader), (m_test_loader, mm_test_loader),
    train, test,
    inner_batch_provider, get_model_path,
    hyperparams,
    device=DEVICE,
    model=da,
    scheduler_func=lambda optimizer: torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
)

epoch: 0, loss: 1.7782831611767622, loss_content: 1.200544746828751, loss_domain: 0.6564822108812736 redundancy: -15.748761640468114  acc_d1: 55.400001525878906 acc_d2: 48.25400161743164 acc_domain: 80.39128875732422

epoch: 1, loss: 1.6576224196125084, loss_content: 0.9810701910878571, loss_domain: 0.6987589488566761 redundancy: -4.441344254453417  acc_d1: 67.33333587646484 acc_d2: 54.314945220947266 acc_domain: 76.6507339477539

epoch: 2, loss: 1.5662967084159314, loss_content: 0.8839863706642473, loss_domain: 0.743635226723174 redundancy: -12.264976864129725  acc_d1: 68.33333587646484 acc_d2: 55.80516052246094 acc_domain: 77.7845687866211

epoch: 3, loss: 1.828864038830072, loss_content: 0.7816102370409899, loss_domain: 1.0768016081460765 redundancy: -5.909561113572456  acc_d1: 74.70000457763672 acc_d2: 59.09697341918945 acc_domain: 75.01111602783203

epoch: 4, loss: 0.934167358237253, loss_content: 0.7496725476123918, loss_domain: 0.2388088807043895 redundancy: -10.862813126872963 